In [31]:
import pandas as pd

bgu = pd.read_csv('./takeoff-merged-VQ-BGU-30s.csv', parse_dates=['reportts'])
bdu = pd.read_csv('./takeoff-merged-VQ-BDU-30s.csv', parse_dates=['reportts']).drop(columns=['file_path'])

train = []
test = []

for data in [bgu, bdu]:
  data = data.sort_values('reportts').drop(columns=['dmusw', 'exswpn', 'reason'])

  data = data.loc[:, ~data.columns.str.contains('stw')]

  train_i = int(len(data) * 75 / 100)

  data_train = data[0:train_i]
  train.append(data_train)
  data_test = data[train_i:]
  test.append(data_test) 

train = pd.concat(train)
test  = pd.concat(test).drop(columns=['egtm'])


In [35]:
train.drop(columns=['egtm']).to_csv('X_train.csv', index=False)

In [38]:
train[['reportts', 'acnum', 'pos', 'egtm']].to_csv('y_train.csv', index=False)

In [37]:
test.to_csv('X_test.csv', index=False)

In [3]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
def train_model(X, y):
  assert len(X) == len(y)
  train_i = int(len(X) * 75 / 100)
  X_train, y_train = X[0:train_i], y[0:train_i]
  X_test, y_test = X[train_i:], y[train_i:]

  model = Ridge(alpha=40)
  model.fit(X_train, y_train)

  predicted_train = model.predict(X_train)
  mse = mean_squared_error(y_train, predicted_train, squared=False)
  mae = mean_absolute_error(y_train, predicted_train)

  predicted_test = model.predict(X_test)
  mse = mean_squared_error(y_test, predicted_test, squared=False)
  mae = mean_absolute_error(y_test, predicted_test)

  return mse, mae, model, predicted_train, predicted_test, train_i, y_test